In [1]:
from google.colab import files

uploaded = files.upload()


Saving etiketlenmis_veri.csv to etiketlenmis_veri.csv


In [3]:
import pandas as pd

df = pd.read_csv("etiketlenmis_veri.csv")  # Dosya adını seninkine göre değiştir
print(df.head())  # İlk 5 satırı göster


       artist                 song_title  \
0  Sezen Aksu          Ablam Aşktan Öldü   
1  Sezen Aksu  Acıtmışım Canını Sevdikçe   
2  Sezen Aksu            Adem Olan Anlar   
3  Sezen Aksu            Adı Bende Saklı   
4  Sezen Aksu                Adı Menekşe   

                                              lyrics   mood  
0  ablam aşktan öldü her şey filmlerdeki gibi old...    aşk  
1  hani yangın yerinde sevmiştim ben seni darmada...  üzgün  
2  ben bu dünyaya bir türlü alışamadım bu yüzden ...   öfke  
3  bölünür sancıyla uykular sığınak değil en kuyt...    aşk  
4  bu şehrin meydanlarında garında rıhtımında sen...  üzgün  


In [4]:
# 'aşk' etiketi olan satırları sil
df = df[df["mood"] != "aşk"]

# Kalan etiketlerin sayısını görelim
print(df["mood"].value_counts())


mood
üzgün     684
mutlu     387
öfke      337
mutlu       4
öfke        2
ügün        1
öfje        1
,           1
öfe         1
Name: count, dtype: int64


In [5]:
# Küçük harfe çevir, boşlukları temizle (önlem olarak)
df["mood"] = df["mood"].str.lower().str.strip()

# Mood düzeltmeleri (manuel eşleme)
duzeltme = {
    "öfje": "öfke",
    "öfe": "öfke",
    "öfke ": "öfke",
    "ügün": "üzgün",
    "üzgün ": "üzgün",
    "mutlu ": "mutlu",
    ",": None  # geçersiz olanları silebiliriz
}

# Etiketleri düzelt
df["mood"] = df["mood"].replace(duzeltme)

# Geçersiz (None olan) etiketleri at
df = df[df["mood"].isin(["mutlu", "üzgün", "öfke"])]

# Güncel dağılımı göster
print(df["mood"].value_counts())


mood
üzgün    685
mutlu    391
öfke     341
Name: count, dtype: int64


In [6]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')

# Türkçe stop words
stop_words = set(stopwords.words('turkish'))

# Temizleme fonksiyonu
def temizle(text):
    text = str(text).lower()  # Küçük harfe çevir
    text = re.sub(r"http\S+", "", text)  # URL'leri kaldır
    text = re.sub(r"[^a-zçğıöşü\s]", "", text)  # Noktalama ve özel karakterleri kaldır
    text = re.sub(r"\s+", " ", text).strip()  # Boşlukları temizle

    # Stop words çıkar
    kelimeler = text.split()
    kelimeler = [k for k in kelimeler if k not in stop_words]
    return " ".join(kelimeler)

# Temizlemeyi uygula
df["clean_lyrics"] = df["lyrics"].apply(temizle)

# Sonuçları kontrol et
df[["lyrics", "clean_lyrics", "mood"]].head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,lyrics,clean_lyrics,mood
1,hani yangın yerinde sevmiştim ben seni darmada...,hani yangın yerinde sevmiştim ben seni darmada...,üzgün
2,ben bu dünyaya bir türlü alışamadım bu yüzden ...,ben dünyaya bir türlü alışamadım yüzden insan ...,öfke
4,bu şehrin meydanlarında garında rıhtımında sen...,şehrin meydanlarında garında rıhtımında sensiz...,üzgün
5,senden daha öncesi yok ki çoktan kaldırdım çat...,senden öncesi yok çoktan kaldırdım çatıya böyl...,üzgün
6,ağlamak güzeldir süzülürken yaşlar gözünden sa...,ağlamak güzeldir süzülürken yaşlar gözünden sa...,üzgün


In [7]:
# Silmek istediğin kelimeleri belirle (küçük harfli olmalı çünkü metin zaten lowercase)
yasak_kelimeler = ["lyrics", "contributor"]

def kelime_temizle(text):
    kelimeler = text.split()
    temiz_kelimeler = [k for k in kelimeler if not any(yasak in k for yasak in yasak_kelimeler)]
    return " ".join(temiz_kelimeler)

# Temizle
df["clean_lyrics"] = df["clean_lyrics"].apply(kelime_temizle)

# Sonuç kontrol
df[["lyrics", "clean_lyrics"]].head()


,lyrics,clean_lyrics
1,hani yangın yerinde sevmiştim ben seni darmada...,hani yangın yerinde sevmiştim ben seni darmada...
2,ben bu dünyaya bir türlü alışamadım bu yüzden ...,ben dünyaya bir türlü alışamadım yüzden insan ...
4,bu şehrin meydanlarında garında rıhtımında sen...,şehrin meydanlarında garında rıhtımında sensiz...
5,senden daha öncesi yok ki çoktan kaldırdım çat...,senden öncesi yok çoktan kaldırdım çatıya böyl...
6,ağlamak güzeldir süzülürken yaşlar gözünden sa...,ağlamak güzeldir süzülürken yaşlar gözünden sa...


In [8]:
from google.colab import files

uploaded = files.upload()

Saving birlesik_veri.csv to birlesik_veri.csv


In [9]:
df2 = pd.read_csv("birlesik_veri.csv")  # Dosya adını seninkine göre değiştir


In [10]:
print("Sütunlar:", df2.columns.tolist())


Sütunlar: ['artist', 'song_title', 'lyrics', 'mood']


In [11]:
df2.head()


,artist,song_title,lyrics,mood
0,Tarkan,Kuzu Kuzu,"Kuzu Kuzu Bak Kırıldı kolum kanadım Olmadı, tu...",mutlu
1,Kenan Doğulu,Çakkıdı,Çakkıdı Buralardan hemen gidesim var Yeniden b...,mutlu
2,Ajda Pekkan,Yaz Yaz Yaz,Yaz Yaz Yaz LyricsDuygular var her zaman kolay...,mutlu
3,Nil Karaibrahimgil,Pırlanta,Pırlanta LyricsParan cebinde kalsın Bırak artı...,mutlu
4,Sezen Aksu,Hadi Bakalım,Hadi Bakalım Yerimiz mi dar yoksa yenimiz mi d...,mutlu


In [12]:
print(df2.isnull().sum())


artist        0
song_title    0
lyrics        0
mood          0
dtype: int64


In [13]:
print("Etiketler ve sayıları:")
print(df2["mood"].value_counts())


Etiketler ve sayıları:
mood
üzgün       731
öfke        575
mutlu       391
mutluluk    200
uzgun        54
Name: count, dtype: int64


In [14]:
# Etiketleri küçük harfe çevir ve boşlukları temizle
df2["mood"] = df2["mood"].str.lower().str.strip()

# Hatalı etiketleri düzelt
etiket_duzeltme = {
    "uzgun": "üzgün",
    "mutluluk": "mutlu"
}

df2["mood"] = df2["mood"].replace(etiket_duzeltme)

# Yalnızca 3 geçerli sınıfı tut
df2 = df2[df2["mood"].isin(["mutlu", "üzgün", "öfke"])]

# Kontrol edelim
print(df2["mood"].value_counts())


mood
üzgün    785
mutlu    591
öfke     575
Name: count, dtype: int64


In [15]:
df2["mood"] = df2["mood"].str.lower().str.strip()
df2["mood"] = df2["mood"].replace(duzeltme)
df2 = df2[df2["mood"].isin(["mutlu", "üzgün", "öfke"])]
df2["clean_lyrics"] = df2["lyrics"].apply(temizle)
df2["clean_lyrics"] = df2["clean_lyrics"].apply(kelime_temizle)


In [18]:
# İki veri setini birleştir
df_birlesik = pd.concat([df, df2], ignore_index=True)

# Son durumu kontrol et
print("Toplam örnek sayısı:", len(df_birlesik))
print("Etiket dağılımı:")
print(df_birlesik["mood"].value_counts())

# Gerekirse ilk birkaç satıra bakalım
df_birlesik.head()


Toplam örnek sayısı: 3368
Etiket dağılımı:
mood
üzgün    1470
mutlu     982
öfke      916
Name: count, dtype: int64


,artist,song_title,lyrics,mood,clean_lyrics
0,Sezen Aksu,Acıtmışım Canını Sevdikçe,hani yangın yerinde sevmiştim ben seni darmada...,üzgün,hani yangın yerinde sevmiştim ben seni darmada...
1,Sezen Aksu,Adem Olan Anlar,ben bu dünyaya bir türlü alışamadım bu yüzden ...,öfke,ben dünyaya bir türlü alışamadım yüzden insan ...
2,Sezen Aksu,Adı Menekşe,bu şehrin meydanlarında garında rıhtımında sen...,üzgün,şehrin meydanlarında garında rıhtımında sensiz...
3,Sezen Aksu,Affet,senden daha öncesi yok ki çoktan kaldırdım çat...,üzgün,senden öncesi yok çoktan kaldırdım çatıya böyl...
4,Sezen Aksu,Ağlamak Güzeldir,ağlamak güzeldir süzülürken yaşlar gözünden sa...,üzgün,ağlamak güzeldir süzülürken yaşlar gözünden sa...


In [19]:
df_birlesik.to_csv("birlesik_veri.csv", index=False)


In [20]:
from google.colab import files
files.download("birlesik_veri.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
# Her sınıf için ayrı DataFrame'ler
uzgun_df = df_birlesik[df_birlesik["mood"] == "üzgün"].sample(n=1000, random_state=42)
mutlu_df = df_birlesik[df_birlesik["mood"] == "mutlu"]
ofke_df = df_birlesik[df_birlesik["mood"] == "öfke"]

# Hepsini birleştir
df_dengeli = pd.concat([uzgun_df, mutlu_df, ofke_df], ignore_index=True)

# Yeni etiket dağılımına bakalım
print(df_dengeli["mood"].value_counts())


mood
üzgün    1000
mutlu     982
öfke      916
Name: count, dtype: int64


In [22]:
from google.colab import files
files.download("birlesik_veri.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>